# Sentimental analysis 2
This notebook is used to test the sentiment analysis pipeline.

In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from transformers import pipeline

In [2]:
# use pipeline sentimental analysis
model = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [3]:
# used to decrease size of chunk
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

# use to process the data with the sentimental model
def batch_predict(model, data, batch_size=128):
    ret = []
    for d in chunks(data, batch_size):
        ret.extend(model(d))
    return ret

In [4]:
import numpy as np

# Function to check the score
# I am not sure about the score that author has calculated.
def pred_and_conf(data):
    # change format to softmax, make everything in [0.33, 0.66] range be predicted as neutral
    preds = batch_predict(model, data)
    pr = np.array([x['score'] if x['label'] == 'POSITIVE' else 1 - x['score'] for x in preds])
    pp = np.zeros((pr.shape[0], 3))
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

In [5]:
suite_path = 'sentiment_dt2.pkl' #define suite path name
suite = TestSuite.from_file(suite_path)

In [6]:
suite.run(pred_and_conf, n=100, overwrite=True) # run the test suite with 500 samples and check score with `pred_and_conf` function

Running single positive words
Predicting 20 examples
Running single negative words
Predicting 20 examples
Running single neutral words
Predicting 20 examples
Running Sentiment-laden words in context
Predicting 100 examples
Running neutral words in context
Predicting 100 examples
Running intensifiers
Predicting 200 examples
Running reducers
Predicting 200 examples
Running add positive phrases
Predicting 1100 examples
Running punctuation
Predicting 205 examples
Running typos
Predicting 200 examples
Running 2 typos
Predicting 200 examples
Running used to, but now
Predicting 100 examples
Running "used to" should reduce
Predicting 200 examples
Running simple negations: negative
Predicting 100 examples
Running simple negations: not negative
Predicting 100 examples
Running simple negations: not neutral is still neutral
Predicting 100 examples
Running simple negations: I thought x was positive, but it was not (should be negative)
Predicting 100 examples
Running simple negations: I thought x wa

In [7]:
suite.summary() #summary the test case

Vocabulary

single positive words
Test cases:      20
Fails (rate):    1 (5.0%)

Example fails:
0.9 0.0 0.1 calm
----


single negative words
Test cases:      20
Fails (rate):    3 (15.0%)

Example fails:
0.0 0.0 1.0 ravages
----
0.0 0.0 1.0 chilling
----
0.0 0.0 1.0 chilled
----


single neutral words
Test cases:      20
Fails (rate):    18 (90.0%)

Example fails:
0.0 0.0 1.0 change
----
0.0 0.0 1.0 regular
----
1.0 0.0 0.0 mild
----


Sentiment-laden words in context
Test cases:      4680
Test cases run:  100
Fails (rate):    5 (5.0%)

Example fails:
0.0 0.0 1.0 We chilled the environment.
----
0.0 0.0 1.0 We ravages this environment.
----
0.0 0.0 1.0 I ravages this environment.
----


neutral words in context
Test cases:      2340
Test cases run:  100
Fails (rate):    92 (92.0%)

Example fails:
0.9 0.0 0.1 This seasonality is variable.
----
1.0 0.0 0.0 The atmosphere was moderate.
----
0.0 0.0 1.0 The weather is seasonal.
----


intensifiers
Test cases:      2000
Test cases run:  10